In [36]:
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from numpy.f2py.auxfuncs import throw_error

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings



### Création de l'authentification API Spotify

In [37]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIFY_CLIENT_ID = "ed9ece4a286d42f1aa834c82bed1f95e"
SPOTIFY_CLIENT_SECRET = "e1f250bb36914f3c9235c88df325eff6"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri="http://localhost:8000/callback",
    scope="user-top-read playlist-read-private user-library-read playlist-modify-public playlist-modify-private"
))

### Get the user's top tracks

In [45]:
top_tracks = sp.current_user_top_tracks(limit=50, time_range='long_term')

print("Your top tracks:")

for idx, track in enumerate(top_tracks['items']):
    print(f"{idx + 1}: {track['name']} by {track['artists'][0]['name']}")
    

Your top tracks:
1: Where Is My Mind? - 2007 Remaster by Pixies
2: Jocelyn Flores by XXXTENTACION
3: Falling Down - Bonus Track by Lil Peep
4: Teardrop by Massive Attack
5: The Night We Met by Lord Huron
6: Évidemment by Lomepal
7: In This Dark Time by Aime Simone
8: death bed (coffee for your head) by Powfu
9: Paradise Circus by Massive Attack
10: The Seed by AURORA
11: Take Me To Church by Hozier
12: Reactor by Woodkid
13: Hasarder by Lomepal
14: 7 Years by Lukas Graham
15: Way down We Go by KALEO
16: San by Orelsan
17: Trop beau by Lomepal
18: Another Love by Tom Odell
19: Rip X by iamjakehill
20: Lucid Dreams by Juice WRLD
21: Where's My Love by SYML
22: Sundance by Népal
23: Sorry by Halsey
24: How to Save a Life by The Fray
25: 505 by Arctic Monkeys
26: Runaway by AURORA
27: Flash by Lomepal
28: Nique les clones, Pt. II by Nekfeu
29: Guns for Hire (from the series Arcane League of Legends) by Woodkid
30: Good News by Mac Miller
31: Jour meilleur by Orelsan
32: Si facile by Casseu

### Get the track IDs to fetch audio features

In [46]:
track_ids = [track['id'] for track in top_tracks['items']]

print(f"Track IDs: {track_ids}")

Track IDs: ['7wCmS9TTVUcIhRalDYFgPy', '7m9OqQk4RVRkw9JJdeAw96', '4jvjzW7Hm0yK4LvvE0Paz9', '67Hna13dNDkZvBpTXRIaOJ', '3hRV0jL3vUpRrcy398teAU', '43yYuYeSOOupBHOkH9pRtF', '2tVJpUamLfNdUI7SiyPIFF', '7eJMfftS33KTjuF7lTsMCx', '2BndJYJQ17UcEeUFJP5JmY', '16tvIGzCNfRLVbm8G39DDo', '3dYD57lRAUcMHufyqn9GcI', '020srIDHnOnzyIhzGTcp7s', '6V40xLcDlYwZpz3oKIVNhL', '0VQjTLPG8l3egUHmIxe265', '0y1QJc3SJVPKJ1OvFmFqe6', '36SHbxigbf4iSCHdQVgy6V', '4WB9OeauHXnGOMUK4s80gM', '7jtQIBanIiJOMS6RyCx6jZ', '0j2BydmlwcMqffV08hFDyI', '285pBltuF7vW8TeWk8hdRR', '1B62o4CbdL9ckGvwsz2cgn', '3CBjC9bcb5MzAnP4Hu9SVL', '44n97yHySt0Z9rqPaXgjCK', '5fVZC9GiM4e8vu99W0Xf6J', '0BxE4FqsDD1Ot4YuBXwAPp', '1WKOnYtM6HtzYvEJlSEIv5', '3EjJfMN67Bcv3FCGR9UgBB', '67obelIiEq3vhpkolCCzZR', '2H3HPGuPyhcbg5AqoQ4BWr', '1DWZUa5Mzf2BwzpHtgbHPY', '6VOUfCrjmxH62oijWgbC6O', '7GfmLz1shp0aMwkPYHan0y', '1VGzxJnVQND7Cg5H5wGj14', '54fap9q6JOH7xNJBEfJch6', '6cT2ovEBYIdVIdjRM532dp', '62w3djsCFX85c5V6qfsI7Y', '5tP7eNuBiAPCElikyAL8h0', '2Ms33RTRCT6gArrpcrPxmo', 

### Load data_appended dataset and fetch the top tracks using ids

In [55]:
data = pd.read_csv("data/data_appended.csv")

top_tracks_data = data[data['id'].isin(track_ids)]

print("Top tracks data :")
print(top_tracks_data)

print("Tracks that are not in top track :")
for idx,track in enumerate(top_tracks['items']):
    if track['id'] not in top_tracks_data['id'].values:
        print(f"{idx}: {track['name']} by {track['artists'][0]['name']}")

Top tracks data :
        valence  year  acousticness                       artists  \
11217    0.6630  1978       0.03840                ['The Police']   
15298    0.2160  1998       0.01730            ['Massive Attack']   
15617    0.1650  2000       0.74800                  ['Coldplay']   
16028    0.2550  2002       0.59900                  ['Coldplay']   
16609    0.3610  2005       0.26900                  ['The Fray']   
18901    0.3370  2016       0.57900                     ['KALEO']   
19006    0.4370  2017       0.46900              ['XXXTENTACION']   
19045    0.2220  2017       0.02750              ['Harry Styles']   
19187    0.3510  2017       0.91300                    ['Halsey']   
19207    0.2180  2018       0.34900                ['Juice WRLD']   
19223    0.2730  2018       0.01750  ['Lil Peep', 'XXXTENTACION']   
19245    0.2040  2018       0.61900                      ['Joji']   
19645    0.3480  2020       0.73100      ['Powfu', 'beabadoobee']   
35061    0.2950 

/var/folders/4v/kbc66d2n3_38tlh4_ggwywxr0000gp/T/ipykernel_17509/2072300795.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/data_appended.csv")


In [40]:
#Throw an error to stop the execution here
throw_error()

TypeError: throw_error.__init__() missing 1 required positional argument: 'mess'

### Create Spotify playlist

In [29]:
# Get user ID (required to create a playlist)
user_id = sp.current_user()['id']

# Create a new playlist
playlist_name = "AI-ify Playlist"
playlist_description = "This is a playlist created using AI-ify."
playlist = sp.user_playlist_create(user_id, playlist_name, public=True, description=playlist_description)

# Add tracks to the playlist
sp.playlist_add_items(playlist['id'], track_ids)

{'snapshot_id': 'AAAAAnOtBCvDWAOs75w0WfvtnizJmFpR'}